In [1]:
import torch
from torchvision import datasets, transforms,models
import torch.utils.data.sampler  as sampler
import torch.utils.data as data
import torch.nn as nn

import numpy as np
import argparse
import random
import os

from custom_datasets import *
import model
import vgg
from solver import Solver
from utils import *
import arguments_yaml
import pickle
import visdom
import shutil
import yaml
from easydict import EasyDict
from initialization import WeightInit
from lib.Utility.utils import save_checkpoint
import visualization
from torchsummary import summary
import GPUtil as GPU

In [2]:

def create_flders(splits,args):
    Flags = {}
    Flags['Dir']='task_net_models/'
    Flags['MNT']=args.out_path+'/'
    Flags['sPath']=Flags['MNT']+args.environment+'/'
    Dir_Use=splits
    savedModels=['imScores','savedModels']


    #remove existing directory
    if os.path.exists(Flags['sPath']) and os.path.isdir(Flags['sPath']):
        shutil.rmtree(Flags['sPath'])
    
    #create the directory
    os.makedirs(Flags['sPath'])
    for directory in Dir_Use:
        if not os.path.exists(Flags['sPath']+str(int(directory*100))):
            os.makedirs(Flags['sPath']+str(int(directory*100)))
            Flags[str(int(directory*100))]=Flags['sPath']+str(int(directory*100))+'/'


    # if not os.path.exists(Flags['sPath']):
    #     os.makedirs(Flags['sPath']+environment)
    return Flags

def config_to_str(config):
    attrs = vars(config)
    string_val = "Config: -----\n"
    string_val += "\n".join("%s: %s" % item for item in attrs.items())
    string_val += "\n----------"
    return string_val

def cifar_transformer():
    return transforms.Compose([
            transforms.ToTensor(),
                # transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
            # transforms.Normalize(mean=[0.5, 0.5, 0.5,],
            #                     std=[0.5, 0.5, 0.5]),
        ])
def caltech_transformer():
    return transforms.Compose([
         transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        # transforms.Scale(256),
        # transforms.CenterCrop(224),
        # transforms.ToTensor(),
        #   transforms.Normalize(mean=[0.485, 0.456, 0.406],
        #                         std=[0.229, 0.224, 0.225])
            # transforms.RandomResizedCrop(224),
            # transforms.ToTensor(),
                # transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
            # transforms.Normalize(mean=[0.5, 0.5, 0.5,],
            #                     std=[0.5, 0.5, 0.5]),
        ])

def plot_visdom(vis,x,y,winName,plotName):
    options = dict(fillarea=False,width=400,height=400,xlabel='Iteration',ylabel='Loss',title=winName)
    if (vis.win_exists(winName)==False):
        win = vis.line(X=np.array([0]),Y=np.array([0]),win=winName,name=plotName,opts=options)
    else:
        vis.line(X=np.array([x]),Y=np.array([y]),win=winName,update='append',name=plotName)

In [3]:
import argparse
import os

def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--work-path', type=str,default='hey')

    args = parser.parse_args()

    # if not os.path.exists(args.out_path):
    #     os.mkdir(args.out_path)
    
    return args


In [4]:
with open('jupyter.yaml') as f:
        config = yaml.load(f)#yaml.load('jupyter.yaml')

/home/jmandivarapu1/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [5]:
#args = get_args()
args = EasyDict(config)
args

{'adversary_param': 'default=1',
 'batch-size': 128,
 'batch_norm': 5e-05,
 'batch_size': 128,
 'beta': 1,
 'cuda': True,
 'data_path': '/mnt/iscsi/data/Jay/datasets/cifar10/',
 'dataset': 'cifar10',
 'device': 'cpu',
 'distance_function': 'cosine',
 'dropout': 0.0,
 'environment': 'CF10_Encoder_Decoder_Jupyter',
 'epochs': 300,
 'joint': True,
 'latent_dim': 25088,
 'learning_rate': 0.001,
 'log_interval': 500,
 'log_name': 'accuracies.log',
 'method': True,
 'model_samples': 1,
 'num_ML_steps': 10,
 'num_adv_steps': 11,
 'num_classes': 10,
 'openset_datasets': 'cifar10,FashionMNIST,AudioMNIST,KMNIST,CIFAR10,CIFAR100,SVHN',
 'openset_weibull_tailsize': 0.05,
 'out_path': '/mnt/iscsi/data/Jay/ActiveLearning',
 'patch_size': 32,
 'pf': 100,
 'print_freq': 250,
 'resume': '',
 'sameIndexs': True,
 'samplePerClass': False,
 'sampler': 'classifierProbability',
 'samplerMethod': 'classifierProbability,LatentMeanDistance,WiebullOutlierProbs,Entropy',
 'train_epochs': 30,
 'train_var': True,


In [6]:


# import the correct loss and training functions depending which model to optimize
# TODO: these could easily be refactored into one function, but we kept it this way for modularity

environment=args.environment
cfg = {"server": "jmandivarapu1@retina.cs.gsu.edu","port": 8097}
vis = visdom.Visdom('http://' + cfg["server"], port = cfg["port"])
vis.delete_env(args.environment) #If you want to clear all the old plots for this python Experiments.Resets the Environment
vis = visdom.Visdom('http://' + cfg["server"], port = cfg["port"],env=environment)

# vis = visdom.Visdom()

# vis.delete_env(environment) #If you want to clear all the old plots for this python Experiments.Resets the Environment
# vis = visdom.Visdom(env=environment)
if args.train_var:
    if args.joint:
        print("came to the Joint Training")
        from lib.Training.train import train_var_joint as train
        from lib.Training.validate import validate_var_joint as validate
        from lib.Training.loss_functions import var_loss_function_joint as criterion
    else:
        print("came to the expected loop")
        from lib.Training.train import train_var as train
        from lib.Training.validate import validate_var as validate
        from lib.Training.loss_functions import var_loss_function as criterion
    from lib.Training.evaluate import eval_var_dataset as evaluate
else:
    if args.joint:
        from lib.Training.train import train_joint as train
        from lib.Training.validate import validate_joint as validate
        from lib.Training.loss_functions import loss_function_joint as criterion
    else:
        from lib.Training.train import train as train
        from lib.Training.validate import validate as validate
        from lib.Training.loss_functions import loss_function as criterion
from lib.OpenSet.meta_recognition import Weibull_Sampler as WieBullSampler



if args.dataset == 'cifar10':
    test_dataloader = data.DataLoader(
            datasets.CIFAR10(args.data_path, download=True, transform=cifar_transformer(), train=False),
        batch_size=args.batch_size, drop_last=False)

    train_dataset = CIFAR10(args.data_path)
    args.num_images = 50000
    args.num_val = 5000
    args.budget = 2500
    args.initial_budget = 5000
    args.num_classes = 10
elif args.dataset == 'cifar100':
    test_dataloader = data.DataLoader(
            datasets.CIFAR100(args.data_path, download=True, transform=cifar_transformer(), train=False),
         batch_size=args.batch_size, drop_last=False)

    train_dataset = CIFAR100(args.data_path)
    print("=========CIFAR 100===============")
    args.num_val = 5000
    args.num_images = 50000
    args.budget = 2500
    args.initial_budget = 5000
    args.num_classes = 100

elif args.dataset == 'imagenet':
    test_dataloader = data.DataLoader(
            datasets.ImageFolder(args.data_path, transform=imagenet_transformer()),
        drop_last=False, batch_size=args.batch_size)

    train_dataset = ImageNet(args.data_path)

    args.num_val = 128120
    args.num_images = 1281167
    args.budget = 64060
    args.initial_budget = 128120
    args.num_classes = 1000
elif args.dataset == 'caltech256':
    print("path os fata is",args.data_path)
    test_dataloader = data.DataLoader(datasets.ImageFolder(args.data_path+'test/',transform=caltech_transformer()),drop_last=True, batch_size=args.test_batch_size)
    train_dataset = Caltech256(args.data_path+'train/')
    print(len(train_dataset),len(test_dataloader))

    # args.num_val = 00
    # args.num_images = 6500
    # args.budget = 1530
    # args.initial_budget = 6000
    # args.num_classes = 256
    args.num_val = 2700
    args.num_images = 26683
    args.budget = 1530
    args.initial_budget = 3060
    args.num_classes = 256
elif args.dataset == 'Cityscapes':
    test_dataloader = data.DataLoader(
            datasets.Cityscapes(args.data_path,download=True, transform=imagenet_transformer()),
        drop_last=False, batch_size=args.batch_size)

    train_dataset = Cityscapes(args.data_path)

    args.num_val = 128120
    args.num_images = 1281167
    args.budget = 64060
    args.initial_budget = 128120
    args.num_classes = 1000
else:
    raise NotImplementedError

if args.sameIndexs:
    print("Setting the random Index")
    random.seed(30)
args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Config: %s" % config_to_str(args))
all_indices = set(np.arange(args.num_images))
val_indices = random.sample(all_indices, args.num_val)
all_indices = np.setdiff1d(list(all_indices), val_indices)
initial_indices = random.sample(list(all_indices), args.initial_budget)
sampler = data.sampler.SubsetRandomSampler(initial_indices)
val_sampler = data.sampler.SubsetRandomSampler(val_indices)

#The below lines are only for weibull distrubution which need two validation sets to pll off
val_indices_set1=val_indices[0:int(len(val_indices)/2)]
val_indices_set2=val_indices[int(len(val_indices)/2):]
val_sampler_set1 = data.sampler.SubsetRandomSampler(val_indices_set1)
val_sampler_set2 = data.sampler.SubsetRandomSampler(val_indices_set2)
val_dataloader_set1 = data.DataLoader(train_dataset, sampler=val_sampler_set1,batch_size=args.batch_size, drop_last=False)
val_dataloader_set2 = data.DataLoader(train_dataset, sampler=val_sampler_set2,batch_size=args.batch_size, drop_last=False)

# dataset with labels available
querry_dataloader = data.DataLoader(train_dataset, sampler=sampler, batch_size=args.batch_size, drop_last=False)
val_dataloader = data.DataLoader(train_dataset, sampler=val_sampler,batch_size=args.batch_size, drop_last=False)

print("length of the Querry loader",len(querry_dataloader)*args.batch_size)
print("length of the Test loader",len(test_dataloader)*args.batch_size)
print("length of the Validation loader",len(val_dataloader)*args.batch_size)
print("length of the Validation loader ",len(val_dataloader_set1)*args.batch_size) 
print("length of the Validation loader",len(val_dataloader_set2)*args.batch_size)  
args.cuda = torch.cuda.is_available()

solver = Solver(args, test_dataloader,val_dataloader_set1,val_dataloader_set2)

splits = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4]

current_indices = list(initial_indices)

accuracies = []

Flags=create_flders(splits,args)
print(Flags)
    
    



Setting up a new session...
Setting up a new session...


came to the Joint Training
Files already downloaded and verified
Files already downloaded and verified
Setting the random Index
Config: Config: -----
cuda: True
device: cuda
workers: 4
patch_size: 32
dropout: 0.0
weight_decay: 5e-05
environment: CF10_Encoder_Decoder_Jupyter
dataset: cifar10
num_classes: 10
log_interval: 500
print_freq: 250
epochs: 300
batch-size: 128
learning_rate: 0.001
batch_norm: 5e-05
pf: 100
num_adv_steps: 11
num_ML_steps: 10
adversary_param: default=1
sameIndexs: True
method: True
sampler: classifierProbability
samplerMethod: classifierProbability,LatentMeanDistance,WiebullOutlierProbs,Entropy
samplePerClass: False
batch_size: 128
train_epochs: 30
latent_dim: 25088
joint: True
train_var: True
weight-init: kaiming-normal
wrn_depth: 16
wrn_widen_factor: 10
wrn_embedding_size: 16
var_latent_dim: 60
var_beta: 0.1
var_samples: 1
model_samples: 1
openset_datasets: cifar10,FashionMNIST,AudioMNIST,KMNIST,CIFAR10,CIFAR100,SVHN
distance_function: cosine
openset_weibull_tai

In [ ]:
task_model=model.WRN_actual(args.device,args.num_classes, 3, args)
task_model.load_state_dict(torch.load('best_74.pt'))
task_model = task_model.cuda()
task_model.train()

unlabeled_indices = np.setdiff1d(list(all_indices), current_indices)
unlabeled_sampler = data.sampler.SubsetRandomSampler(unlabeled_indices)
unlabeled_dataloader = data.DataLoader(train_dataset, 
            sampler=unlabeled_sampler, batch_size=args.batch_size, drop_last=False)

In [ ]:
from lib.OpenSet.meta_recognition import Weibull_Sampler_all_datasets as WieBullSampler

In [7]:
import torch
import numpy as np
import libmr
import collections
import visualization
import lib.Datasets.datasets as datasets


def sample(values,all_indices,method,budget):

    if method!='NONE':
        all_preds = values
        all_indices = all_indices
        # all_preds = torch.stack(all_preds)
        all_preds = all_preds.view(-1)
        # need to multiply by -1 to be able to use torch.topk 
        all_preds *= -1

        # select the points which the discriminator things are the most likely to be unlabeled
        _, querry_indices = torch.topk(all_preds, int(budget))
        querry_pool_indices = np.asarray(all_indices)[querry_indices]

        return querry_pool_indices
    else:
        return all_indices
def get_means(tensors_list):
    """
    Calculate the mean of a list of tensors for each tensor in the list. In our case the list typically contains
    a tensor for each class, such as the per class z values.

    Parameters:
        tensors_list (list): List of Tensors

    Returns:
        list: List of Tensors containing mean vectors
    """

    means = []
    for i in range(len(tensors_list)):
        if isinstance(tensors_list[i], torch.Tensor):
            means.append(torch.mean(tensors_list[i], dim=0))
        else:
            means.append([])

    return means


def calc_distances_to_means(means, tensors, distance_function='cosine'):
    """
    Function to calculate distances between tensors, in our case the mean zs per class and z for each input.
    Wrapper around torch.nn.functonal distances with specification of which distance function to choose.

    Parameters:
        means (list): List of length corresponding to number of classes containing torch tensors (typically mean zs).
        tensors (list): List of length corresponding to number of classes containing tensors (typically zs).
        distance_function (str): Specification of distance function. Choice of cosine|euclidean|mix.

    Returns:
        list: List of length corresponding to number of classes containing tensors with distance values
    """

    def distance_func(a, b, w_eucl, w_cos):
        """
        Weighted distance function consisting of cosine and euclidean components.

        Parameters:
            a (torch.Tensor): First tensor.
            b (torch.Tensor): Second tensor.
            w_eucl (float): Weight for the euclidean distance term.
            w_cos (float): Weight for the cosine similarity term.
        """
        d = w_cos * (1 - torch.nn.functional.cosine_similarity(a.view(1, -1), b)) + \
            w_eucl * torch.nn.functional.pairwise_distance(a.view(1, -1), b, p=2)
        return d

    distances = []

    # weight values for individual distance components
    w_eucl = 0.0
    w_cos = 0.0
    if distance_function == 'euclidean':
        w_eucl = 1.0
    elif distance_function == 'cosine':
        w_cos = 1.0
    elif distance_function == 'mix':
        w_eucl = 0.5
        w_cos = 0.5
    else:
        raise ValueError("distance function not implemented")

    # loop through each class in means and calculate the distances with the respective tensor.
    for i in range(len(means)):
        # check for tensor type, e.g. list could be empty
        if isinstance(tensors[i], torch.Tensor) and isinstance(means[i], torch.Tensor):
            distances.append(distance_func(means[i], tensors[i], w_eucl, w_cos))
        else:
            distances.append([])

    return distances


def fit_weibull_models(distribution_values, tailsizes, num_max_fits=50):
    """
    Function to fit weibull models on distribution values per class. The distribution values in our case are the
    distances of an inputs approximate posterior value to the per class mean latent z, i.e. The Weibull model fits
    regions of high density and gives credible intervals.
    The tailsize specifies how many outliers are expected in the dataset for which the model has been trained.
    We use libmr https://github.com/Vastlab/libMR (installable through e.g. pip) for the Weibull model fitting.

    Parameters:
        distribution_values (list): Values on which the fit is conducted. In our case latent space distances.
        tailsizes (list): List of integers, specifying tailsizes per class. For a balanced dataset typically the same.
        num_max_fits (int): Number of attempts to fit the Weibull models before timing out and returning unsuccessfully.

    Returns:
        list: List of Weibull models with their respective parameters (stored in libmr class instances).
    """

    weibull_models = []

    # loop through the list containing distance values per class
    for i in range(len(distribution_values)):
        # for each class set the initial success to False and number of attempts to 0
        is_valid = False
        count = 0

        # If the list contains distance values conduct a fit. If it is empty, e.g. because there is not a single
        # prediction for the corresponding class, continue with the next class. Note that the latter isn't expected for
        # a model that has been trained for even just a short while.
        if isinstance(distribution_values[i], torch.Tensor):
            distribution_values[i] = distribution_values[i].cpu().numpy()
            # weibull model per class
            weibull_models.append(libmr.MR())
            # attempt num_max_fits many fits before aborting
            while is_valid is False and count < num_max_fits:
                # conduct the fit with libmr
                weibull_models[i].fit_high(distribution_values[i], tailsizes[i])
                is_valid = weibull_models[i].is_valid
                count += 1
            if not is_valid:
                print("Weibull fit for class " + str(i) + " not successful after " + str(num_max_fits) + " attempts")
                return weibull_models, False
        else:
            weibull_models.append([])

    return weibull_models, True


def calc_outlier_probs(weibull_models, distances):
    """
    Calculates statistical outlier probability using the weibull models' CDF.

    Note that we have coded this function to loop over each class because we have previously categorized the distances
    into their respective classes already.

    Parameters:
        weibull_models (list): List of libmr class instances containing the Weibull model parameters and functions.
        distances (list): List of per class torch tensors or numpy arrays with latent space distance values.

    Returns:
        list: List of length corresponding to number of classes with outlier probabilities for each respective input.
    """

    outlier_probs = []
    # loop through all classes, i.e. all available weibull models as there is one weibull model per class.
    for i in range(len(weibull_models)):
        # optionally convert the type of the distance vectors
        if isinstance(distances[i], torch.Tensor):
            distances[i] = distances[i].cpu().numpy().astype(np.double)
        elif isinstance(distances[i], list):
            # empty list
            outlier_probs.append([])
            continue
        else:
            distances[i] = distances[i].astype(np.double)

        # use the Weibull models' CDF to evaluate statistical outlier rejection probabilities.
        outlier_probs.append(weibull_models[i].w_score_vector(distances[i]))

    return outlier_probs


def calc_openset_classification(data_outlier_probs, num_classes, num_outlier_threshs=50):
    """
    Calculates the percentage of dataset outliers given a set of outlier probabilities over a range of rejection priors.

    Parameters:
         data_outlier_probs (list): List of outlier probabilities for an entire dataset, categorized by class.
         num_classes (int): Number of classes.
         num_outlier_threshs (int): Number of outlier rejection priors (evenly spread over the interval (0,1)).

    Returns:
        dict: Dictionary containing outlier percentages and corresponding rejection prior values.
    """

    dataset_outliers = []
    threshs = []

    # loop through each rejection prior value and evaluate the percentage of the dataset being considered as
    # statistical outliers, i.e. each data point's outlier probability > rejection prior.
    for i in range(num_outlier_threshs - 1):
        outlier_threshold = (i + 1) * (1.0 / num_outlier_threshs)
        threshs.append(outlier_threshold)

        dataset_outliers.append(0)
        total_dataset = 0

        for j in range(num_classes):
            total_dataset += len(data_outlier_probs[j])

            for k in range(len(data_outlier_probs[j])):
                if data_outlier_probs[j][k] > outlier_threshold:
                    dataset_outliers[i] += 1

        dataset_outliers[i] = dataset_outliers[i] / float(total_dataset)

    return {"thresholds": threshs, "outlier_percentage": dataset_outliers}


def calc_entropy_classification(dataset_entropies, max_thresh_value,args, num_outlier_threshs=50):
    """
    Calculates the percentage of dataset outliers given a set of entropies over a range of rejection priors.
    Parameters:
         dataset_entropies (list): List of entropies for the entire dataset (each instance)
         num_outlier_threshs (int): Number of outlier rejection priors (evenly spread over the interval (0,1)).
    Returns:
        dict: Dictionary containing outlier percentages and corresponding rejection prior values.
    """

    dataset_outliers = []
    threshs = []

    total_dataset = float(len(dataset_entropies))

    # loop through each rejection prior value and evaluate the percentage of the dataset being considered as
    # statistical outliers, i.e. each data point's outlier probability > rejection prior.
    for i in range(num_outlier_threshs - 1):
        outlier_threshold = (i + 1) * (max_thresh_value / num_outlier_threshs)
        threshs.append(outlier_threshold)

        dataset_outliers.append(0)

        for k in range(len(dataset_entropies)):
            if dataset_entropies[k] > outlier_threshold:
                dataset_outliers[i] += 1

        dataset_outliers[i] = dataset_outliers[i] / total_dataset

    return {"entropy_thresholds": threshs, "entropy_outlier_percentage": dataset_outliers}


def eval_var_openset_dataset(model, data_loader, num_classes, device,args, latent_var_samples=1, model_var_samples=1):
    """
    Evaluates an entire dataset with the variational or joint model and stores z values, latent mus and sigmas and
    output predictions such that they can later be used for statistical outlier evaluation with the fitted Weibull
    models. This is merely for convenience to keep the rest of the code API the same. Note that the Weibull model's
    prediction of whether a sample from an unknown dataset is a statistical outlier or not can be done on an instance
    level. Similar to the eval_dataset function but without splitting of correct vs. false predictions as the dataset
    is unknown in the open-set scenario.
    Parameters:
        model (torch.nn.module): Trained model.
        data_loader (torch.utils.data.DataLoader): The dataset loader.
        num_classes (int): Number of classes.
        device (str): Device to compute on.
        latent_var_samples (int): Number of latent space variational samples.
        model_var_samples (int): Number of variational samples of the entire model, e.g. MC dropout.
    Returns:
        dict: Dictionary of results and latent values.
    """

    # switch to evaluation mode unless MC dropout is active
    if model_var_samples > 1:
        model.train()
    else:
        model.eval()

    out_mus = []
    out_sigmas = []
    encoded_mus = []
    encoded_sigmas = []
    zs = []
    all_preds = []
    all_indices = []

    out_entropy = []
    collect_indexes_per_class=[]

    for i in range(num_classes):
        out_mus.append([])
        out_sigmas.append([])
        encoded_mus.append([])
        encoded_sigmas.append([])
        zs.append([])
        collect_indexes_per_class.append([])

    # evaluate the encoder and classifier and store results in corresponding lists according to predicted class.
    # Prediction mean confidence and uncertainty is also obtained if amount of latent samples is greater than one.
    with torch.no_grad():
        for inputs, classes,indexes in data_loader:
            inputs, classes = inputs.to(device), classes.to(device)

            out_samples = torch.zeros(model_var_samples, latent_var_samples, inputs.size(0), num_classes).to(device)
            z_samples = torch.zeros(model_var_samples, latent_var_samples,
                                    inputs.size(0), model.latent_dim).to(device)

            # sampling the model, then z and classifying
            for k in range(model_var_samples):
                encoded_mu, encoded_std,_ = model.encode(inputs)

                for i in range(latent_var_samples):
                    z = model.reparameterize(encoded_mu, encoded_std)
                    z_samples[k][i] = z

                    cl = model.classifier(z)
                    out = torch.nn.functional.softmax(cl, dim=1)
                    out_samples[k][i] = out

            # calculate the mean and std. Only removes a dummy dimension if number of variational samples is set to one.
            out_mean = torch.mean(torch.mean(out_samples, dim=0), dim=0)
            # preds = out_mean.cpu().data
            all_preds.extend(out_mean.cpu().data)
            all_indices.extend(indexes)
            if model_var_samples > 1:
                out_std = torch.std(torch.mean(out_samples, dim=0), dim=0)
            else:
                out_std = torch.squeeze(torch.std(out_samples, dim=1))
            zs_mean = torch.mean(torch.mean(z_samples, dim=0), dim=0)
            
            # calculate entropy for the means of samples: - sum pc*log(pc)
            eps = 1e-10
            out_entropy.append(- torch.sum(out_mean*torch.log(out_mean + eps), dim=1).cpu().detach().numpy())

            # In contrast to the eval_dataset function, there is no split into correct or false values as the dataset
            # is unknown.
            for i in range(inputs.size(0)):
                idx = torch.argmax(out_mean[i]).item()
                out_mus[idx].append(out_mean[i][idx].item())
                out_sigmas[idx].append(out_std[i][idx].item())
                encoded_mus[idx].append(encoded_mu[i].data)
                encoded_sigmas[idx].append(encoded_std[i].data)
                zs[idx].append(zs_mean[i].data)
                collect_indexes_per_class[idx].append(indexes[i].item())

    # stack latent activations into a tensor
    for i in range(len(encoded_mus)):
        if len(encoded_mus[i]) > 0:
            encoded_mus[i] = torch.stack(encoded_mus[i], dim=0)
            encoded_sigmas[i] = torch.stack(encoded_sigmas[i], dim=0)
            zs[i] = torch.stack(zs[i], dim=0)

    out_entropy = np.concatenate(out_entropy).ravel().tolist()

    d={"encoded_mus": encoded_mus, "encoded_sigmas": encoded_sigmas,
            "out_mus": out_mus, "out_sigmas": out_sigmas, "zs": zs,
            "out_entropy": out_entropy,'all_preds':all_preds,    "all_indices":all_indices}

    # with open('outfile', 'wb') as fp:pickle.dump(d, fp)

    all_preds=[]
    for i in range(0,len(d['all_preds'])):
        idx=torch.argmax(d['all_preds'][i]).item()
        all_preds.append(d['all_preds'][i][idx])#.data.item())
    all_preds
    all_preds = torch.stack(all_preds)
    all_preds = all_preds.view(-1)
    # need to multiply by -1 to be able to use torch.topk 
    all_preds *= -1

    # select the points which the discriminator things are the most likely to be unlabeled
    _, querry_indices = torch.topk(all_preds, int(args.budget))
    querry_pool_indices = np.asarray(all_indices)[querry_indices]

    # Return a dictionary of stored values.
    # with open('outfile', 'wb') as fp:pickle.dump(d, fp)
    return {"encoded_mus": encoded_mus, "encoded_sigmas": encoded_sigmas,
            "out_mus": out_mus, "out_sigmas": out_sigmas, "zs": zs,
            "out_entropy": out_entropy,'all_preds':all_preds,
            "all_indices":all_indices,
            'querry_pool_indices':querry_pool_indices,
            'collect_indexes_per_class':collect_indexes_per_class}

def Weibull_Sampler_all_datasets(model,train_loader,test_dataloader,val_loader,unlabeled_dataloader,val_dataloader_set1,val_dataloader_set2,eval_var_dataset,args,save_path):

    dataset_eval_dict_train = eval_var_dataset(model, train_loader,args.num_classes, args.device,latent_var_samples=args.var_samples, model_var_samples=args.model_samples)
    print("Training accuracy: ", dataset_eval_dict_train["accuracy"])#"accuracy"])
    #Start Preparing for the sampling
    # Get the mean of z for correctly classified data inputs
    mean_zs = get_means(dataset_eval_dict_train["zs_correct"])
    # visualize the mean z vectors
    #mean_zs_tensor = torch.stack(mean_zs, dim=0)
    # visualize_means(mean_zs_tensor, num_classes, args.dataset, save_path, "z")
    # calculate each correctly classified example's distance to the mean z
    distances_to_z_means_correct_train = calc_distances_to_means(mean_zs, dataset_eval_dict_train["zs_correct"],args.distance_function)

    #Weibull fitting
    # set tailsize according to command line parameters (according to percentage of dataset size)
    tailsize = int(len(train_loader)*128 * args.openset_weibull_tailsize / args.num_classes)
    print("Fitting Weibull models with tailsize: " + str(tailsize),len(train_loader))
    tailsizes = [tailsize] * args.num_classes
    weibull_models, valid_weibull = fit_weibull_models(distances_to_z_means_correct_train, tailsizes)
    # ------------------------------------------------------------------------------------------
    # Fitting on train dataset complete. Determine rejection thresholds/priors on the created split set
    # ------------------------------------------------------------------------------------------
    print("Evaluating original threshold split dataset: " + args.dataset + ". This may take a while...")
    threshset_eval_dict = eval_var_dataset(model, val_dataloader_set1, args.num_classes, args.device,latent_var_samples=args.var_samples, model_var_samples=args.model_samples)

    # Again calculate distances to mean z
    print("Split set accuracy: ", threshset_eval_dict["accuracy"])
    distances_to_z_means_threshset = calc_distances_to_means(mean_zs, threshset_eval_dict["zs_correct"],args.distance_function)
    # get Weibull outlier probabilities for thresh set
    outlier_probs_threshset = calc_outlier_probs(weibull_models, distances_to_z_means_threshset)
    threshset_classification = calc_openset_classification(outlier_probs_threshset, args.num_classes,num_outlier_threshs=100)
    #print("threshset_classification is",threshset_classification)
    # also check outlier detection based on entropy
    max_entropy = np.max(threshset_eval_dict["out_entropy"])
    print("Max entopy is",max_entropy)
    threshset_entropy_classification = calc_entropy_classification(threshset_eval_dict["out_entropy"],max_entropy,args,num_outlier_threshs=100)
    #print("calc_entropy_classification",threshset_entropy_classification)
    # determine rejection priors based on 5% of the split data considered as inlying
    if (np.array(threshset_classification["outlier_percentage"]) <= 0.05).any() == True:
        EVT_prior_index = np.argwhere(np.array(threshset_classification["outlier_percentage"])<= 0.05)[0][0]
        EVT_prior = threshset_classification["thresholds"][EVT_prior_index]
    else:
        EVT_prior = 0.5
        EVT_prior_index = 50

    if (np.array(threshset_entropy_classification["entropy_outlier_percentage"]) <= 0.05).any() == True:
        entropy_threshold_index = np.argwhere(np.array(threshset_entropy_classification["entropy_outlier_percentage"])
                                            <= 0.05)[0][0]
        entropy_threshold = threshset_entropy_classification["entropy_thresholds"][entropy_threshold_index]
    else:
        # this should never actually happen
        entropy_threshold = np.median(threshset_entropy_classification["entropy_thresholds"])
        entropy_threshold_index = 50
    
    
    # ------------------------------------------------------------------------------------------
    # We evaluate the validation set to later evaluate trained dataset's statistical inlier/outlier estimates.
    print("Evaluating original validation dataset: " + args.dataset + ". This may take a while...")
    dataset_eval_dict = eval_var_dataset(model, val_dataloader_set2, args.num_classes, args.device,latent_var_samples=args.var_samples, model_var_samples=args.model_samples)

    # Again calculate distances to mean z
    print("Validation accuracy: ", dataset_eval_dict["accuracy"])
    distances_to_z_means_correct = calc_distances_to_means(mean_zs, dataset_eval_dict["zs_correct"],args.distance_function)

    # Evaluate outlier probability of trained dataset's validation set
    outlier_probs_correct = calc_outlier_probs(weibull_models, distances_to_z_means_correct)

    dataset_classification_correct = calc_openset_classification(outlier_probs_correct, args.num_classes,num_outlier_threshs=100)
    dataset_entropy_classification_correct = calc_entropy_classification(dataset_eval_dict["out_entropy"],max_entropy,args,num_outlier_threshs=100)

    print(args.dataset + '(trained) EVT outlier percentage: ' +str(dataset_classification_correct["outlier_percentage"][EVT_prior_index]))
    print(args.dataset + '(trained) entropy outlier percentage: ' +str(dataset_entropy_classification_correct["entropy_outlier_percentage"][entropy_threshold_index]))
    
    ##########################################################################################################################################
    #                           START ON THE RESt OF TRAINING SET
    ##########################################################################################################################################
    od=0
    openset_dataset='cifar10'
    openset_datasets_names = args.openset_datasets.strip().split(',')
    openset_sampler_methods=args.samplerMethod.strip().split(',')
    samplerMethod=args.sampler
    print("The Sampling Method is",samplerMethod)
    openset_datasets = []
    for openset_dataset in openset_datasets_names:
        openset_data_init_method = getattr(datasets, openset_dataset)
        openset_datasets.append(openset_data_init_method(torch.cuda.is_available(), args))
    # Repeat process for open set recognition on unseen datasets (
    openset_dataset_eval_dicts = collections.OrderedDict()
    openset_outlier_probs_dict = collections.OrderedDict()
    openset_classification_dict = collections.OrderedDict()
    openset_entropy_classification_dict = collections.OrderedDict()
    topk=[]
    for od, openset_dataset in enumerate(openset_datasets):
        if openset_datasets_names[od]=='cifar10':
            num_classes=10
        elif openset_datasets_names[od]=='CIFAR100':
            num_classes=100
        print("Evaluating openset dataset: " + openset_datasets_names[od] + ". This may take a while...",openset_dataset)
        print("Evaluating on rest of the tain set This may take a while...",len(test_dataloader)*128)
        print("Evaluating openset dataset: " + openset_datasets_names[od] + ". This may take a while...")
        openset_dataset_eval_dict = eval_var_openset_all_datasets(model, openset_dataset.val_loader, num_classes,
                                                         args.device,args, latent_var_samples=args.var_samples,
                                                         model_var_samples=args.model_samples)

        openset_distances_to_z_means = calc_distances_to_means(mean_zs, openset_dataset_eval_dict["zs"],
                                                               args.distance_function)

        openset_outlier_probs = calc_outlier_probs(weibull_models, openset_distances_to_z_means)

        # getting outlier classification accuracies across the entire datasets
        openset_classification = calc_openset_classification(openset_outlier_probs, num_classes,
                                                             num_outlier_threshs=100)

        openset_entropy_classification = calc_entropy_classification(openset_dataset_eval_dict["out_entropy"],
                                                                     max_entropy,args, num_outlier_threshs=100)

        # dictionary of dictionaries: per datasetname one dictionary with respective values
        openset_dataset_eval_dicts[openset_datasets_names[od]] = openset_dataset_eval_dict
        openset_outlier_probs_dict[openset_datasets_names[od]] = openset_outlier_probs
        openset_classification_dict[openset_datasets_names[od]] = openset_classification
        openset_entropy_classification_dict[openset_datasets_names[od]] = openset_entropy_classification

    # print outlier rejection values for all unseen unknown datasets
    for other_data_name, other_data_dict in openset_classification_dict.items():
        print(other_data_name + ' EVT outlier percentage: ' +
              str(other_data_dict["outlier_percentage"][entropy_threshold_index]))

    for other_data_name, other_data_dict in openset_entropy_classification_dict.items():
        print(other_data_name + ' entropy outlier percentage: ' +
              str(other_data_dict["entropy_outlier_percentage"][entropy_threshold_index]))

    # joint prediction uncertainty plot for all datasets
    if (args.train_var and args.var_samples > 1) or args.model_samples > 1:
        visualize_classification_uncertainty(dataset_eval_dict["out_mus_correct"],
                                             dataset_eval_dict["out_sigmas_correct"],
                                             openset_dataset_eval_dicts,
                                             "out_mus", "out_sigmas",
                                             args.dataset + ' (trained)',
                                             args.var_samples, save_path)

    # visualize the outlier probabilities
    visualization.visualize_weibull_outlier_probabilities(outlier_probs_correct, openset_outlier_probs_dict,
                                            args.dataset + ' (trained)', save_path, tailsize)

    visualization.visualize_classification_scores(dataset_eval_dict["out_mus_correct"], openset_dataset_eval_dicts, 'out_mus',
                                    args.dataset + ' (trained)', save_path)

    visualization.visualize_entropy_histogram(dataset_eval_dict["out_entropy"], openset_dataset_eval_dicts,
                                dataset_entropy_classification_correct["entropy_thresholds"][-1], "out_entropy",
                                args.dataset + ' (trained)', save_path)

    # joint plot for outlier detection accuracy for seen and both unseen datasets
    visualization.visualize_openset_classification(dataset_classification_correct["outlier_percentage"],
                                     openset_classification_dict, "outlier_percentage",
                                     args.dataset + ' (trained)',
                                     dataset_classification_correct["thresholds"], save_path, tailsize)

    visualization.visualize_entropy_classification(dataset_entropy_classification_correct["entropy_outlier_percentage"],
                                     openset_entropy_classification_dict, "entropy_outlier_percentage",
                                     args.dataset + ' (trained)',
                                     dataset_entropy_classification_correct["entropy_thresholds"], save_path)

    return topk



def eval_var_openset_all_datasets(model, data_loader, num_classes, device,args, latent_var_samples=1, model_var_samples=1):
    """
    Evaluates an entire dataset with the variational or joint model and stores z values, latent mus and sigmas and
    output predictions such that they can later be used for statistical outlier evaluation with the fitted Weibull
    models. This is merely for convenience to keep the rest of the code API the same. Note that the Weibull model's
    prediction of whether a sample from an unknown dataset is a statistical outlier or not can be done on an instance
    level. Similar to the eval_dataset function but without splitting of correct vs. false predictions as the dataset
    is unknown in the open-set scenario.
    Parameters:
        model (torch.nn.module): Trained model.
        data_loader (torch.utils.data.DataLoader): The dataset loader.
        num_classes (int): Number of classes.
        device (str): Device to compute on.
        latent_var_samples (int): Number of latent space variational samples.
        model_var_samples (int): Number of variational samples of the entire model, e.g. MC dropout.
    Returns:
        dict: Dictionary of results and latent values.
    """

    # switch to evaluation mode unless MC dropout is active
    if model_var_samples > 1:
        model.train()
    else:
        model.eval()

    out_mus = []
    out_sigmas = []
    encoded_mus = []
    encoded_sigmas = []
    zs = []
    all_preds = []
    all_indices = []

    out_entropy = []
    collect_indexes_per_class=[]

    for i in range(num_classes):
        out_mus.append([])
        out_sigmas.append([])
        encoded_mus.append([])
        encoded_sigmas.append([])
        zs.append([])
        collect_indexes_per_class.append([])

    # evaluate the encoder and classifier and store results in corresponding lists according to predicted class.
    # Prediction mean confidence and uncertainty is also obtained if amount of latent samples is greater than one.
    with torch.no_grad():
        for inputs, classes in data_loader:
            inputs, classes = inputs.to(device), classes.to(device)

            out_samples = torch.zeros(model_var_samples, latent_var_samples, inputs.size(0), num_classes).to(device)
            z_samples = torch.zeros(model_var_samples, latent_var_samples,
                                    inputs.size(0), model.latent_dim).to(device)

            # sampling the model, then z and classifying
            for k in range(model_var_samples):
                encoded_mu, encoded_std,_ = model.encode(inputs)

                for i in range(latent_var_samples):
                    z = model.reparameterize(encoded_mu, encoded_std)
                    z_samples[k][i] = z

                    cl = model.classifier(z)
                    out = torch.nn.functional.softmax(cl, dim=1)
                    out_samples[k][i] = out

            # calculate the mean and std. Only removes a dummy dimension if number of variational samples is set to one.
            out_mean = torch.mean(torch.mean(out_samples, dim=0), dim=0)
            # preds = out_mean.cpu().data
            all_preds.extend(out_mean.cpu().data)
            #all_indices.extend(indexes)
            if model_var_samples > 1:
                out_std = torch.std(torch.mean(out_samples, dim=0), dim=0)
            else:
                out_std = torch.squeeze(torch.std(out_samples, dim=1))
            zs_mean = torch.mean(torch.mean(z_samples, dim=0), dim=0)
            
            # calculate entropy for the means of samples: - sum pc*log(pc)
            eps = 1e-10
            out_entropy.append(- torch.sum(out_mean*torch.log(out_mean + eps), dim=1).cpu().detach().numpy())

            # In contrast to the eval_dataset function, there is no split into correct or false values as the dataset
            # is unknown.
            for i in range(inputs.size(0)):
                idx = torch.argmax(out_mean[i]).item()
                out_mus[idx].append(out_mean[i][idx].item())
                out_sigmas[idx].append(out_std[i][idx].item())
                encoded_mus[idx].append(encoded_mu[i].data)
                encoded_sigmas[idx].append(encoded_std[i].data)
                zs[idx].append(zs_mean[i].data)
                #collect_indexes_per_class[idx].append(indexes[i].item())

    # stack latent activations into a tensor
    for i in range(len(encoded_mus)):
        if len(encoded_mus[i]) > 0:
            encoded_mus[i] = torch.stack(encoded_mus[i], dim=0)
            encoded_sigmas[i] = torch.stack(encoded_sigmas[i], dim=0)
            zs[i] = torch.stack(zs[i], dim=0)

    out_entropy = np.concatenate(out_entropy).ravel().tolist()

    d={"encoded_mus": encoded_mus, "encoded_sigmas": encoded_sigmas,
            "out_mus": out_mus, "out_sigmas": out_sigmas, "zs": zs,
            "out_entropy": out_entropy,'all_preds':all_preds,    "all_indices":all_indices}

    # with open('outfile', 'wb') as fp:pickle.dump(d, fp)

    all_preds=[]
    for i in range(0,len(d['all_preds'])):
        idx=torch.argmax(d['all_preds'][i]).item()
        all_preds.append(d['all_preds'][i][idx])#.data.item())
    all_preds
    all_preds = torch.stack(all_preds)
    all_preds = all_preds.view(-1)
    # need to multiply by -1 to be able to use torch.topk 
    all_preds *= -1

    # select the points which the discriminator things are the most likely to be unlabeled
    _, querry_indices = torch.topk(all_preds, int(args.budget))
    #querry_pool_indices = np.asarray(all_indices)[querry_indices]

    # Return a dictionary of stored values.
    # with open('outfile', 'wb') as fp:pickle.dump(d, fp)
    return {"encoded_mus": encoded_mus, "encoded_sigmas": encoded_sigmas,
            "out_mus": out_mus, "out_sigmas": out_sigmas, "zs": zs,
            "out_entropy": out_entropy,'all_preds':all_preds,
            "all_indices":all_indices,
            # 'querry_pool_indices':querry_pool_indices,
            'collect_indexes_per_class':collect_indexes_per_class}

In [54]:
z=[]
def Weibull_Sampler_(model,train_loader,test_dataloader,val_loader,unlabeled_dataloader,val_dataloader_set1,val_dataloader_set2,eval_var_dataset,args,save_path):

    dataset_eval_dict_train = eval_var_dataset(model, train_loader,args.num_classes, args.device,latent_var_samples=args.var_samples, model_var_samples=args.model_samples)
    
    print("Training accuracy: ", dataset_eval_dict_train["accuracy"])#"accuracy"])
    #Start Preparing for the sampling
    # Get the mean of z for correctly classified data inputs
    mean_zs = get_means(dataset_eval_dict_train["zs_correct"])
    # visualize the mean z vectors
    #mean_zs_tensor = torch.stack(mean_zs, dim=0)
    # visualize_means(mean_zs_tensor, num_classes, args.dataset, save_path, "z")
    # calculate each correctly classified example's distance to the mean z
    distances_to_z_means_correct_train = calc_distances_to_means(mean_zs, dataset_eval_dict_train["zs_correct"],args.distance_function)

    #Weibull fitting
    # set tailsize according to command line parameters (according to percentage of dataset size)
#     tailsize = int(len(train_loader)*128 * args.openset_weibull_tailsize / args.num_classes)
#     print("Fitting Weibull models with tailsize: " + str(tailsize),len(train_loader))
#     tailsizes = [tailsize] * args.num_classes

    tailsizes=[]
    for i in range(0,len(dataset_eval_dict_train["zs_correct"])):
        tailsizes.append(int(0.05*len(dataset_eval_dict_train["zs_correct"][i])))
    weibull_models, valid_weibull = fit_weibull_models(distances_to_z_means_correct_train, tailsizes)
    # ------------------------------------------------------------------------------------------
    # Fitting on train dataset complete. Determine rejection thresholds/priors on the created split set
    # ------------------------------------------------------------------------------------------
    print("Evaluating original threshold split dataset: " + args.dataset + ". This may take a while...")
    threshset_eval_dict = eval_var_dataset(model, val_dataloader_set1, args.num_classes, args.device,latent_var_samples=args.var_samples, model_var_samples=args.model_samples)

    # Again calculate distances to mean z
    print("Split set accuracy: ", threshset_eval_dict["accuracy"])
    distances_to_z_means_threshset = calc_distances_to_means(mean_zs, threshset_eval_dict["zs_correct"],args.distance_function)
    # get Weibull outlier probabilities for thresh set
    outlier_probs_threshset = calc_outlier_probs(weibull_models, distances_to_z_means_threshset)
    threshset_classification = calc_openset_classification(outlier_probs_threshset, args.num_classes,num_outlier_threshs=100)
    #print("threshset_classification is",threshset_classification)
    # also check outlier detection based on entropy
    max_entropy = np.max(threshset_eval_dict["out_entropy"])
    print("Max entopy is",max_entropy)
    threshset_entropy_classification = calc_entropy_classification(threshset_eval_dict["out_entropy"],max_entropy,args,num_outlier_threshs=100)
    #print("calc_entropy_classification",threshset_entropy_classification)
    # determine rejection priors based on 5% of the split data considered as inlying
    if (np.array(threshset_classification["outlier_percentage"]) <= 0.05).any() == True:
        EVT_prior_index = np.argwhere(np.array(threshset_classification["outlier_percentage"])<= 0.05)[0][0]
        EVT_prior = threshset_classification["thresholds"][EVT_prior_index]
    else:
        EVT_prior = 0.5
        EVT_prior_index = 50

    if (np.array(threshset_entropy_classification["entropy_outlier_percentage"]) <= 0.05).any() == True:
        entropy_threshold_index = np.argwhere(np.array(threshset_entropy_classification["entropy_outlier_percentage"])
                                            <= 0.05)[0][0]
        entropy_threshold = threshset_entropy_classification["entropy_thresholds"][entropy_threshold_index]
    else:
        # this should never actually happen
        entropy_threshold = np.median(threshset_entropy_classification["entropy_thresholds"])
        entropy_threshold_index = 50
    
    
    # ------------------------------------------------------------------------------------------
    # We evaluate the validation set to later evaluate trained dataset's statistical inlier/outlier estimates.
    print("Evaluating original validation dataset: " + args.dataset + ". This may take a while...")
    dataset_eval_dict = eval_var_dataset(model, val_dataloader_set2, args.num_classes, args.device,latent_var_samples=args.var_samples, model_var_samples=args.model_samples)

    # Again calculate distances to mean z
    print("Validation accuracy: ", dataset_eval_dict["accuracy"])
    distances_to_z_means_correct = calc_distances_to_means(mean_zs, dataset_eval_dict["zs_correct"],args.distance_function)

    # Evaluate outlier probability of trained dataset's validation set
    outlier_probs_correct = calc_outlier_probs(weibull_models, distances_to_z_means_correct)

    dataset_classification_correct = calc_openset_classification(outlier_probs_correct, args.num_classes,num_outlier_threshs=100)
    dataset_entropy_classification_correct = calc_entropy_classification(dataset_eval_dict["out_entropy"],max_entropy,args,num_outlier_threshs=100)

    print(args.dataset + '(trained) EVT outlier percentage: ' +str(dataset_classification_correct["outlier_percentage"][EVT_prior_index]))
    print(args.dataset + '(trained) entropy outlier percentage: ' +str(dataset_entropy_classification_correct["entropy_outlier_percentage"][entropy_threshold_index]))
    
    ##########################################################################################################################################
    #                           START ON THE RESt OF TRAINING SET
    ##########################################################################################################################################
    od=0
    openset_dataset='cifar10'
    openset_datasets_names = args.openset_datasets.strip().split(',')
    openset_sampler_methods=args.samplerMethod.strip().split(',')
    samplerMethod=args.sampler
    print("The Sampling Method is",samplerMethod)
    openset_datasets = []
    # Repeat process for open set recognition on unseen datasets (
    openset_dataset_eval_dicts = collections.OrderedDict()
    openset_outlier_probs_dict = collections.OrderedDict()
    openset_classification_dict = collections.OrderedDict()
    openset_entropy_classification_dict = collections.OrderedDict()
    print("Evaluating on rest of the tain set This may take a while...",len(test_dataloader)*128)
    openset_dataset_eval_dict = eval_var_openset_dataset(model,unlabeled_dataloader, args.num_classes,
                                                        args.device,args, latent_var_samples=args.var_samples,
                                                        model_var_samples=args.model_samples)
    


    #with open('openset_dataset_eval_dict', 'wb') as fp:pickle.dump(openset_dataset_eval_dict, fp)
    #with open('mean_zs', 'wb') as fp:pickle.dump(mean_zs, fp)
    # with open('weibull_models', 'wb') as fp:pickle.dump(weibull_models, fp)
    #with open('openset_distances_to_z_means', 'wb') as fp:pickle.dump(openset_distances_to_z_means, fp)
    # sys.exit()
    openset_distances_to_z_means = calc_distances_to_means(mean_zs, openset_dataset_eval_dict["zs"],'cosine')#args.distance_function)

    openset_outlier_probs = calc_outlier_probs(weibull_models, openset_distances_to_z_means)

    # getting outlier classification accuracies across the entire datasets
    openset_classification = calc_openset_classification(openset_outlier_probs, args.num_classes,num_outlier_threshs=100)

    openset_entropy_classification = calc_entropy_classification(openset_dataset_eval_dict["out_entropy"],max_entropy, args,num_outlier_threshs=100)
    
    return openset_outlier_probs,openset_classification,openset_entropy_classification,openset_dataset_eval_dict

    if samplerMethod=='classifierProbability':
        topk=sample([],openset_dataset_eval_dict['querry_pool_indices'],'NONE',args.budget)
    elif  samplerMethod=='LatentMeanDistance':
        Rvalues=[]
        Rindexes=[]
        for i in range(0,args.num_classes):#openset_outlier_probs:
            Rvalues.append(torch.Tensor(openset_distances_to_z_means[i]))
            Rindexes.append(torch.Tensor(openset_dataset_eval_dict['collect_indexes_per_class'][i]))
        if  args.samplePerClass:
            print("came to sample per class")
            perclassSample=[]
            for i in range(0,args.num_classes):
                topk=sample(Rvalues[i],Rindexes[i],samplerMethod,int(args.budget/args.num_classes))
                perclassSample.append(torch.Tensor(topk))

            topk=torch.cat(perclassSample,dim=0)
        else:
            topk=sample(torch.cat(Rvalues, dim=0),torch.cat(Rindexes, dim=0),samplerMethod,args.budget)
            
    elif  samplerMethod=='WiebullOutlierProbs':
        Rvalues=[]
        Rindexes=[]
        for i in range(0,args.num_classes):#openset_outlier_probs:
            Rvalues.append(torch.Tensor(openset_outlier_probs[i]))
            Rindexes.append(torch.Tensor(openset_dataset_eval_dict['collect_indexes_per_class'][i]))
        if  args.samplePerClass:
            print("came to  per class")
            perclassSample=[]
            for i in range(0,args.num_classes):
                topk=sample(Rvalues[i],Rindexes[i],samplerMethod,int(args.budget/args.num_classes))
                perclassSample.append(torch.Tensor(topk))

            topk=torch.cat(perclassSample,dim=0)
        else:
            topk=sample(torch.cat(Rvalues, dim=0),torch.cat(Rindexes, dim=0),samplerMethod,args.budget)
            
    elif samplerMethod=='Entropy':
        topk=sample(openset_dataset_eval_dict,openset_classification,samplerMethod)
    elif samplerMethod=='openSet':
        topk=sample(openset_dataset_eval_dict,openset_entropy_classification,samplerMethod)

    # dictionary of dictionaries: per datasetname one dictionary with respective values
    openset_dataset_eval_dicts[openset_datasets_names[od]] = openset_dataset_eval_dict
    openset_outlier_probs_dict[openset_datasets_names[od]] = openset_outlier_probs
    openset_classification_dict[openset_datasets_names[od]] = openset_classification
    openset_entropy_classification_dict[openset_datasets_names[od]] = openset_entropy_classification

    # print outlier rejection values for all unseen unknown datasets
    for other_data_name, other_data_dict in openset_classification_dict.items():
        print(other_data_name + ' EVT outlier percentage: ' +
            str(other_data_dict["outlier_percentage"][entropy_threshold_index]))

    for other_data_name, other_data_dict in openset_entropy_classification_dict.items():
        print(other_data_name + ' entropy outlier percentage: ' +
            str(other_data_dict["entropy_outlier_percentage"][entropy_threshold_index]))
        

    # joint prediction uncertainty plot for all datasets
    if (args.train_var and args.var_samples > 1) or args.model_samples > 1:
        visualization.visualize_classification_uncertainty(dataset_eval_dict["out_mus_correct"],
                                            dataset_eval_dict["out_sigmas_correct"],
                                            openset_dataset_eval_dicts,
                                            "out_mus", "out_sigmas",
                                            args.dataset + ' (trained)',
                                            args.var_samples, save_path)

    # visualize the outlier probabilities
    visualization.visualize_weibull_outlier_probabilities(outlier_probs_correct, openset_outlier_probs_dict,
                                            args.dataset + ' (trained)', save_path, tailsize)

    visualization.visualize_classification_scores(dataset_eval_dict["out_mus_correct"], openset_dataset_eval_dicts, 'out_mus',
                                    args.dataset + ' (trained)', save_path)

    visualization.visualize_entropy_histogram(dataset_eval_dict["out_entropy"], openset_dataset_eval_dicts,
                                dataset_entropy_classification_correct["entropy_thresholds"][-1], "out_entropy",
                                args.dataset + ' (trained)', save_path)

    # joint plot for outlier detection accuracy for seen and both unseen datasets
    visualization.visualize_openset_classification(dataset_classification_correct["outlier_percentage"],
                                    openset_classification_dict, "outlier_percentage",
                                    args.dataset + ' (trained)',
                                    dataset_classification_correct["thresholds"], save_path, tailsize)

    visualization.visualize_entropy_classification(dataset_entropy_classification_correct["entropy_outlier_percentage"],
                                    openset_entropy_classification_dict, "entropy_outlier_percentage",
                                    args.dataset + ' (trained)',
                                    dataset_entropy_classification_correct["entropy_thresholds"], save_path)

    return topk

In [56]:
for split in splits:
    num_colors=3
    iterations=0
    epoch = 0
    best_prec = 0
    best_acc=0
    best_loss = random.getrandbits(128)
    lr_change=[150,250]
    #task_model=model.WRN(args.device,args.num_classes, num_colors, args)
    if args.dataset == 'caltech256':
        task_model=model.WRN_caltech_actual(args.device,args.num_classes, num_colors, args)#models.vgg16(pretrained=True)#
        # for param in task_model.parameters():
        #     param.requires_grad = False
        #     # n_inputs = 25088#task_model.classifier[6].in_features
        #     # Add on classifier
        #     task_model.classifier[6] = nn.Sequential(
        #         nn.Linear(4096, 256), nn.ReLU(), nn.Dropout(0.2),
        #         nn.Linear(256, 256), nn.LogSoftmax(dim=1))
    else:
        task_model=model.WRN_actual(args.device,args.num_classes, num_colors, args)
    #print("mode",task_model)

    task_model.train()
    # task_model.load_state_dict(torch.load('save_path/best_0.pt'))
    if args.cuda:
        task_model = task_model.cuda()
        #task_model = torch.nn.DataParallel(task_model).to(args.device)
    #summary(task_model, (3, 32, 32))
    # WeightInitializer = WeightInit(args.weight_init)
    # WeightInitializer.init_model(task_model)
    unlabeled_indices = np.setdiff1d(list(all_indices), current_indices)
    unlabeled_sampler = data.sampler.SubsetRandomSampler(unlabeled_indices)
    unlabeled_dataloader = data.DataLoader(train_dataset, 
            sampler=unlabeled_sampler, batch_size=args.batch_size, drop_last=False)
    task_model.load_state_dict(torch.load('best_74.pt'))
    save_path=Flags[str(int(split*100))]
    openset_outlier_probs,openset_classification,openset_entropy_classification,openset_dataset_eval_dict=Weibull_Sampler_(task_model,querry_dataloader,test_dataloader,val_dataloader,unlabeled_dataloader,val_dataloader_set1,val_dataloader_set2,evaluate,args,save_path)
    print("length od the Unlabled loader",len(unlabeled_dataloader)*128)  
    
    sys.exit()
    optimizer = torch.optim.Adam(task_model.parameters(), lr=args.learning_rate, weight_decay=args.weight_decay)
    while epoch < args.epochs:
        if (epoch in lr_change):
            for param in optimizer.param_groups:
                param['lr'] = param['lr'] / 10
        # train
        finished_iter,acc,loss=train(querry_dataloader,
                validate,
                test_dataloader,
                task_model, 
                criterion, 
                epoch, 
                optimizer,
                vis, 
                args.device, 
                args,
                split,
                iterations)

        # evaluate on validation set
        # acc, loss = validate(, task_model, criterion, epoch, vis, args.device, args)
        iterations=finished_iter

        # increment epoch counters
        epoch += 1

                # remember best prec@1 and save checkpoint
        is_best = best_acc < acc
        if is_best:
            best_acc=acc
            save_path=Flags[str(int(split*100))]
            best_model=task_model
            torch.save(task_model.state_dict(),os.path.join(Flags[str(int(split*100))],"best_"+str(int(best_acc))+".pt"))
            save_checkpoint({'epoch': epoch,
                            'state_dict': task_model.state_dict(),
                            'best_prec': best_prec,
                            'best_loss': best_loss,
                            'optimizer': optimizer.state_dict()},
                            is_best, save_path)
    accuracies.append(best_acc)
    print("All accuracies until now",accuracies)
    GPUs = GPU.getGPUs()
    for gpu in GPUs:print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
    with open(save_path+'acc_'+str(int(len(accuracies))), 'wb') as fp:pickle.dump(accuracies, fp)
    print('Final accuracy with {}% of data is: {:.2f}'.format(int(split*100), acc))
    sampled_indices=WieBullSampler(best_model,querry_dataloader,test_dataloader,val_dataloader,unlabeled_dataloader,val_dataloader_set1,val_dataloader_set2,evaluate,args,save_path)
    current_indices = list(current_indices) + list(sampled_indices)
    sampler = data.sampler.SubsetRandomSampler(current_indices)
    querry_dataloader = data.DataLoader(train_dataset, sampler=sampler, 
            batch_size=args.batch_size, drop_last=True)


Training accuracy:  0.9822
Evaluating original threshold split dataset: cifar10. This may take a while...
Split set accuracy:  0.7056
Max entopy is 2.0256171226501465
Evaluating original validation dataset: cifar10. This may take a while...
Validation accuracy:  0.7348
cifar10(trained) EVT outlier percentage: 0.05389221556886228
cifar10(trained) entropy outlier percentage: 0.0464
The Sampling Method is classifierProbability
Evaluating on rest of the tain set This may take a while... 10112
length od the Unlabled loader 40064


NameError: name 'sys' is not defined

In [ ]:
openset_outlier_probs,openset_classification,openset_entropy_classification

In [51]:
dataset_eval_dict_train

{'accuracy': 0.9828,
 'encoded_mus_correct': [tensor([[-0.1665, -0.5866, -0.8325,  ...,  1.4434, -0.6495, -1.0462],
          [-0.2000, -0.5364, -0.8055,  ...,  1.3936, -0.5232, -1.0005],
          [-0.1518, -0.5714, -0.8120,  ...,  1.4102, -0.7147, -0.9975],
          ...,
          [-0.1531, -0.5860, -0.8678,  ...,  1.4170, -0.7114, -1.0396],
          [-0.1826, -0.4766, -0.5744,  ...,  1.6220, -0.5607, -0.8116],
          [-0.1570, -0.5900, -0.8624,  ...,  1.4453, -0.6932, -1.0531]],
         device='cuda:0'),
  tensor([[ 1.2414, -0.3129, -0.5245,  ..., -0.6715, -1.1175, -0.0719],
          [ 1.1538, -0.2902, -0.5071,  ..., -0.5392, -1.0460, -0.1182],
          [ 1.2601, -0.2853, -0.5381,  ..., -0.7297, -0.9524, -0.0969],
          ...,
          [ 1.1769, -0.2522, -0.4965,  ..., -0.7324, -0.7730, -0.1021],
          [ 1.0437, -0.2495, -0.4753,  ..., -0.3659, -0.8533, -0.1292],
          [ 1.2640, -0.2791, -0.5308,  ..., -0.7597, -0.9326, -0.0914]],
         device='cuda:0'),
  tens

In [50]:
openset_outlier_probs

[array([0.18845286, 0.11114424, 0.22911643, ..., 0.16455347, 0.0485969 ,
        0.08146869]),
 array([0.3108196 , 0.02076598, 0.99964149, ..., 0.01979476, 0.01365904,
        0.00699612]),
 array([0.99999735, 1.        , 0.28508157, ..., 0.0396625 , 0.10326041,
        0.00781564]),
 array([1.68042195e-03, 6.15128655e-03, 1.09042791e-03, ...,
        4.22144483e-05, 6.29267975e-04, 4.17405602e-02]),
 array([0.3691157 , 0.46528941, 0.08164648, ..., 0.01543642, 0.78307321,
        0.90121556]),
 array([4.98441276e-03, 4.62790922e-05, 1.93862987e-04, ...,
        4.76528561e-04, 2.85861685e-04, 9.88850072e-01]),
 array([1.        , 0.0196557 , 0.18357401, ..., 0.02546236, 0.03294075,
        0.01296844]),
 array([0.36060218, 0.01288553, 0.01696941, ..., 0.01267575, 0.00447282,
        0.14722637]),
 array([1.05879423e-03, 1.69134161e-02, 1.42751796e-02, ...,
        1.70981755e-04, 2.26939588e-04, 1.00000000e+00]),
 array([0.29520956, 0.15599163, 0.02946072, ..., 0.13483195, 0.09353115,


In [25]:
sampled_indices.keys()
dataset_eval_dict_train=sampled_indices
dataset_eval_dict_train.keys()

dict_keys(['accuracy', 'encoded_mus_correct', 'encoded_mus_false', 'encoded_sigmas_correct', 'encoded_sigmas_false', 'zs_correct', 'zs_false', 'out_mus_correct', 'out_sigmas_correct', 'out_mus_false', 'out_sigmas_false', 'out_entropy'])

In [58]:
openset_dataset_eval_dict['collect_indexes_per_class']

[[13660,
  17082,
  14090,
  30710,
  41300,
  39819,
  21001,
  3063,
  26178,
  9799,
  6617,
  2898,
  22350,
  41345,
  43924,
  31202,
  45314,
  38885,
  47689,
  1980,
  44709,
  19740,
  2221,
  49528,
  36887,
  14239,
  19761,
  39530,
  9257,
  42083,
  43708,
  10470,
  6090,
  32635,
  28068,
  33134,
  34611,
  45188,
  38855,
  9281,
  11352,
  47780,
  29468,
  47431,
  10526,
  40956,
  21639,
  6711,
  31201,
  7053,
  37797,
  663,
  4543,
  3066,
  15590,
  3359,
  39666,
  37772,
  4689,
  44350,
  48520,
  15581,
  1144,
  18925,
  40343,
  28883,
  7879,
  35996,
  22622,
  11008,
  5322,
  35201,
  40542,
  11506,
  26879,
  30440,
  16511,
  38576,
  34645,
  41457,
  5523,
  30257,
  16139,
  3361,
  33998,
  7127,
  21803,
  49205,
  27231,
  20954,
  42205,
  38921,
  23319,
  42456,
  2034,
  30280,
  13710,
  39959,
  29118,
  16191,
  16143,
  37377,
  44470,
  33944,
  39747,
  9694,
  18203,
  23629,
  16146,
  1878,
  3263,
  3665,
  12599,
  17179,
  

In [53]:
dataset_eval_dict_train

{'accuracy': 0.9828,
 'encoded_mus_correct': [tensor([[-0.1665, -0.5866, -0.8325,  ...,  1.4434, -0.6495, -1.0462],
          [-0.2000, -0.5364, -0.8055,  ...,  1.3936, -0.5232, -1.0005],
          [-0.1518, -0.5714, -0.8120,  ...,  1.4102, -0.7147, -0.9975],
          ...,
          [-0.1531, -0.5860, -0.8678,  ...,  1.4170, -0.7114, -1.0396],
          [-0.1826, -0.4766, -0.5744,  ...,  1.6220, -0.5607, -0.8116],
          [-0.1570, -0.5900, -0.8624,  ...,  1.4453, -0.6932, -1.0531]],
         device='cuda:0'),
  tensor([[ 1.2414, -0.3129, -0.5245,  ..., -0.6715, -1.1175, -0.0719],
          [ 1.1538, -0.2902, -0.5071,  ..., -0.5392, -1.0460, -0.1182],
          [ 1.2601, -0.2853, -0.5381,  ..., -0.7297, -0.9524, -0.0969],
          ...,
          [ 1.1769, -0.2522, -0.4965,  ..., -0.7324, -0.7730, -0.1021],
          [ 1.0437, -0.2495, -0.4753,  ..., -0.3659, -0.8533, -0.1292],
          [ 1.2640, -0.2791, -0.5308,  ..., -0.7597, -0.9326, -0.0914]],
         device='cuda:0'),
  tens

In [108]:
Rvalues=[]
Rindexes=[]
for i in range(0,args.num_classes):#openset_outlier_probs:
    Rvalues.append(torch.Tensor(openset_outlier_probs[i]))
    Rindexes.append(torch.Tensor(openset_dataset_eval_dict['collect_indexes_per_class'][i]))

In [109]:
Rvalues

[tensor([0.0798, 0.1433, 0.2626,  ..., 0.0637, 0.0034, 1.0000]),
 tensor([0.1586, 0.0009, 0.0020,  ..., 0.4877, 0.0127, 0.0679]),
 tensor([0.0007, 0.0204, 0.0056,  ..., 0.0092, 0.0193, 0.0837]),
 tensor([0.0222, 1.0000, 0.4063,  ..., 0.0506, 0.3472, 1.0000]),
 tensor([0.0296, 0.2065, 0.0439,  ..., 0.9982, 0.0869, 0.3312]),
 tensor([0.0064, 0.0669, 0.0610,  ..., 0.3825, 1.0000, 0.5702]),
 tensor([0.5048, 0.0418, 0.0022,  ..., 0.0013, 0.1285, 0.0660]),
 tensor([0.5400, 0.0048, 0.0012,  ..., 0.2332, 0.0086, 0.0081]),
 tensor([0.0017, 0.0196, 0.0034,  ..., 0.1259, 0.0014, 0.0011]),
 tensor([0.0258, 0.7933, 0.0412,  ..., 0.0128, 0.0097, 0.0172])]

In [91]:
arr=torch.cat(Rvalues, dim=0).data.numpy()
numpy.where(arr == numpy.amin(arr)) #numpy.amin(torch.cat(Rvalues, dim=0).data.numpy())

(array([32712]),)

In [110]:
all_preds=torch.cat(Rvalues, dim=0)
all_preds *= -1
_,querry_indices=torch.topk(all_preds, int(2500))
querry_indices

tensor([32712, 34997, 24729,  ..., 12143, 28803,  4704])

In [99]:
Rindexes

[tensor([13660., 17082., 14090.,  ..., 19493., 33819., 33034.]),
 tensor([19341.,  6984., 46177.,  ...,  3143., 35805., 21418.]),
 tensor([34497., 37528., 10155.,  ..., 30688., 18322., 26807.]),
 tensor([ 4102., 13735., 15247.,  ..., 49070., 37151., 46443.]),
 tensor([ 1880.,  5409., 45420.,  ..., 49407., 10609., 12021.]),
 tensor([42654., 25774., 44632.,  ...,  1979.,   835., 12135.]),
 tensor([10260., 48103., 49929.,  ..., 19707.,  4734., 41491.]),
 tensor([ 2821., 28528., 43270.,  ...,  6195., 11460.,  5336.]),
 tensor([23258., 21028., 44501.,  ..., 18660., 39502., 37534.]),
 tensor([20181., 41478., 30776.,  ..., 44588., 10827.,  9704.])]

In [111]:
np.asarray(torch.cat(Rindexes, dim=0).data.numpy())[querry_indices]

array([15092.,  4507.,   424., ..., 48384., 36079., 25619.], dtype=float32)

In [78]:
torch.cat(Rindexes, dim=0)

tensor([13660., 17082., 14090.,  ..., 44588., 10827.,  9704.])

In [96]:
_, querry_indices=sample(torch.cat(Rvalues, dim=0),torch.cat(Rindexes, dim=0),'LatentMeanDistance',args.budget)
querry_indices    

ValueError: too many values to unpack (expected 2)

In [26]:
 mean_zs = get_means(dataset_eval_dict_train["zs_correct"])
# calculate each correctly classified example's distance to the mean z
distances_to_z_means_correct_train = calc_distances_to_means(mean_zs, dataset_eval_dict_train["zs_correct"],args.distance_function)
#Weibull fitting
tailsizes=[]
for i in range(0,len(dataset_eval_dict_train["zs_correct"])):
    tailsizes.append(int(0.05*len(dataset_eval_dict_train["zs_correct"][i])))
weibull_models, valid_weibull = fit_weibull_models(distances_to_z_means_correct_train, tailsizes)


# ------------------------------------------------------------------------------------------
# Fitting on train dataset complete. Determine rejection thresholds/priors on the created split set
# ------------------------------------------------------------------------------------------
print("Evaluating original threshold split dataset: " + args.dataset + ". This may take a while...")
threshset_eval_dict = eval_var_dataset(model, val_dataloader_set1, args.num_classes, args.device,latent_var_samples=args.var_samples, model_var_samples=args.model_samples)

# Again calculate distances to mean z
print("Split set accuracy: ", threshset_eval_dict["accuracy"])
distances_to_z_means_threshset = calc_distances_to_means(mean_zs, threshset_eval_dict["zs_correct"],args.distance_function)
# get Weibull outlier probabilities for thresh set
outlier_probs_threshset = calc_outlier_probs(weibull_models, distances_to_z_means_threshset)
threshset_classification = calc_openset_classification(outlier_probs_threshset, args.num_classes,num_outlier_threshs=100)
#print("threshset_classification is",threshset_classification)
# also check outlier detection based on entropy
max_entropy = np.max(threshset_eval_dict["out_entropy"])
print("Max entopy is",max_entropy)
threshset_entropy_classification = calc_entropy_classification(threshset_eval_dict["out_entropy"],max_entropy,args,num_outlier_threshs=100)
#print("calc_entropy_classification",threshset_entropy_classification)
# determine rejection priors based on 5% of the split data considered as inlying
if (np.array(threshset_classification["outlier_percentage"]) <= 0.05).any() == True:
    EVT_prior_index = np.argwhere(np.array(threshset_classification["outlier_percentage"])<= 0.05)[0][0]
    EVT_prior = threshset_classification["thresholds"][EVT_prior_index]
else:
    EVT_prior = 0.5
    EVT_prior_index = 50

if (np.array(threshset_entropy_classification["entropy_outlier_percentage"]) <= 0.05).any() == True:
    entropy_threshold_index = np.argwhere(np.array(threshset_entropy_classification["entropy_outlier_percentage"])
                                        <= 0.05)[0][0]
    entropy_threshold = threshset_entropy_classification["entropy_thresholds"][entropy_threshold_index]
else:
    # this should never actually happen
    entropy_threshold = np.median(threshset_entropy_classification["entropy_thresholds"])
    entropy_threshold_index = 50

In [13]:
s=0
tail_sizes=[]
for i in range(0,len(sampled_indices["zs_correct"])):
    print(len(sampled_indices["zs_correct"][i]))
    s=s+len(sampled_indices["zs_correct"][i])
    tail_sizes.append(int(0.05*len(sampled_indices["zs_correct"][i])))
s

486
502
494
478
487
449
521
493
505
495


4910

In [14]:
tail_sizes

[24, 25, 24, 23, 24, 22, 26, 24, 25, 24]

In [30]:
dataset_outliers = []
threshs = []
num_outlier_threshs=100
# loop through each rejection prior value and evaluate the percentage of the dataset being considered as
# statistical outliers, i.e. each data point's outlier probability > rejection prior.
for i in range(num_outlier_threshs - 1):
    outlier_threshold = (i + 1) * (1.0 / num_outlier_threshs)
    threshs.append(outlier_threshold)

    dataset_outliers.append(0)
    total_dataset = 0

    for j in range(num_classes):
        total_dataset += len(data_outlier_probs[j])

        for k in range(len(data_outlier_probs[j])):
            if data_outlier_probs[j][k] > outlier_threshold:
                dataset_outliers[i] += 1

    dataset_outliers[i] = dataset_outliers[i] / float(total_dataset)

# return {"thresholds": threshs, "outlier_percentage": dataset_outliers}
threshs

[0.01,
 0.02,
 0.03,
 0.04,
 0.05,
 0.06,
 0.07,
 0.08,
 0.09,
 0.1,
 0.11,
 0.12,
 0.13,
 0.14,
 0.15,
 0.16,
 0.17,
 0.18,
 0.19,
 0.2,
 0.21,
 0.22,
 0.23,
 0.24,
 0.25,
 0.26,
 0.27,
 0.28,
 0.29,
 0.3,
 0.31,
 0.32,
 0.33,
 0.34,
 0.35000000000000003,
 0.36,
 0.37,
 0.38,
 0.39,
 0.4,
 0.41000000000000003,
 0.42,
 0.43,
 0.44,
 0.45,
 0.46,
 0.47000000000000003,
 0.48,
 0.49,
 0.5,
 0.51,
 0.52,
 0.53,
 0.54,
 0.55,
 0.56,
 0.5700000000000001,
 0.58,
 0.59,
 0.6,
 0.61,
 0.62,
 0.63,
 0.64,
 0.65,
 0.66,
 0.67,
 0.68,
 0.6900000000000001,
 0.7000000000000001,
 0.71,
 0.72,
 0.73,
 0.74,
 0.75,
 0.76,
 0.77,
 0.78,
 0.79,
 0.8,
 0.81,
 0.8200000000000001,
 0.8300000000000001,
 0.84,
 0.85,
 0.86,
 0.87,
 0.88,
 0.89,
 0.9,
 0.91,
 0.92,
 0.93,
 0.9400000000000001,
 0.9500000000000001,
 0.96,
 0.97,
 0.98,
 0.99]

In [65]:
x=[1,2,3,5]

In [73]:
a = [-2,1,5,3,8,5,6]
b = [1,2,5]
c = [ a[i] for i in b]

TypeError: list indices must be integers or slices, not tuple

In [102]:
def plot_sampled_images_class(vis,all_saved_indicies,split,sampled_indices):
    #Getting the statistics of the entire dataset
    count_idx_target=[]
    for index in sampled_indices:
        count_idx_target.append(all_saved_indicies[index])
    counter=collections.Counter(count_idx_target)
    x=collections.OrderedDict(sorted(counter.items()))
    a=np.array(x.keys())
    b=list(x.values())
    acc_options = dict(fillarea=True,width=400,height=400,xlabel='Classes',ylabel='Per Class Count',title=str('Sampler Class Analysis'))
    if split:
        vis.bar(X=b,win='Sampled',opts=acc_options)
    else:
        vis.bar(X=b,win='Sampled_'+str(int(current_split*100)),opts=acc_options)

In [103]:
all_saved_indicies=list(unlabeled_dataloader.dataset.cifar10.targets)

In [106]:
len(querry_indices)

2500

In [107]:
plot_sampled_images_class(vis,all_saved_indicies,0.1,querry_indices)